# Análise de inadimplentes (space net)

## Contexto do desafio

Os responsáveis da Space Net ISP solicitaram a equipe de TI que eles oferececem um plano de marketing para aumentar o ticket médio da empresa, antes de efetuar esse planejamento um dos funcionários se voluntariou para criar uma análise em cima dos dados desses clientes que apresentam essa inadimplência

### O que a empresa precisa fazer para resolver isso?!

1. Importar a base de dados
2. Tratar os dados irrelevantes para a análise
3. Comparar e criar visualizações para facilitar a tomada de decisão

#### Primeira parte do desafio
Como não sei ainda como que faz importações via SQL, eu tive que utilizar o google sheets para recolher as informações dos clientes que entraram em débito com a empresa, porém prestando atenção em duplicatas, não importa quantas vezes eles entrou como inadimplente.

Adquiri uma lista de nomes e id's que relacionam esses clientes ao nosso banco de dados no mês de junho (Data que está sendo feita a análise), irei importar essa lista em csv para o python recolher

In [59]:
import numpy as np
import pandas as pd
import openpyxl

# passo 1 importar as bases de dados
clientes_inadimplentes = pd.read_csv("clientes_inadimplentes-junho.txt")
informacoes_dos_clientes = pd.read_excel("todos_os_clientes.xlsx")

### Excluindo o que não importa
Muitas das vezes, você vai se deparar com um monte de informações que são ou inúteis, eu inválidas para a análise, no caso da Space net o problema se trata da falta de informação sobre os clientes que contratam o serviço, então segue o exemplo de dados úteis que não foram preenchidos:
- Vendedor
- Midia

In [60]:
# passo 2 Tratar os dados
# 2.1 -> Excluir as colunas/horizontais vazias e sem valor para a análise
colunas_vazias = ["Nome Fantasia","smartcard","Linha","gateway id","Fidelidade","Vendedor","Midia","RG Emissor","IP"]
colunas_irrelevantes = ["CPF/CNPJ","Cliente ID","Data Cadastro cliente","Celulares","Telefones","Emails","Login","Senha","Número","Complemento","Ponto de referencia","RG","CEP","Cidade","UF","MAC","Switch","Porta Switch","Tipo"]
informacoes_dos_clientes = informacoes_dos_clientes.drop(colunas_vazias, axis=1)
informacoes_dos_clientes = informacoes_dos_clientes.drop(colunas_irrelevantes, axis=1)

### O que é essa condição?
Essa "condição" existe pois foi a maneira que eu encontrei de interligar/Indexar essas duas listas diferentes, **"Clientes_inadimplentes"** com **"informacoes_dos_clientes"**.
Então eu pesquisei por essa lógica na internet e descobri a função "isin" que iria traduzir para "Está em"
a variável "condicao", que é um objeto do tipo Series contendo valores booleanos, será utilizado para filtrar as linhas correspondentes da 

**clientes_inadimplentes['ID']**

In [66]:
# 2.2 -> Comparação entre duas listas
condicao = informacoes_dos_clientes['Contrato ID'].isin(clientes_inadimplentes['ID'])
informacoes_dos_clientes = informacoes_dos_clientes[condicao]

### A Função da Função
De costume no nosso setor trabalhamos para retirar as informações atraves de "Contagens" entregando assim uma vizualização numérica dos ítens.
Então criei a função, na qual é só inserir o nome da coluna que ele vai entregar a contagem e demonstrar quantos % aquele valor significa para a análise.

In [84]:
# 3ª Parte - Contagem de todas as informações restantes
def contagem_planilha(coluna):
    contagem = informacoes_dos_clientes[coluna].value_counts()
    total_observacoes = contagem.sum()
    print(f"Segue a contagem de {coluna}: ")
    for valor, contagem in contagem.items():
        contagem = print("{}: {} = {:.2%}".format(valor,contagem,(contagem/total_observacoes)))
    print("\n")
    

contagem_planilha("Bairro")
contagem_planilha("Plano Valor")
contagem_planilha("Vencimento")
contagem_planilha("Acres.Desc.")
contagem_planilha("Plano")

Segue a contagem de Bairro: 
BARRO VERMELHO: 237 = 25.79%
SANTA AMÉLIA: 224 = 24.37%
PARQUE SÃO BERNARDO: 125 = 13.60%
HITERLAND: 111 = 12.08%
VILA MAIA: 20 = 2.18%
VILA VIÇOSA: 19 = 2.07%
SÃO BERNARDO: 17 = 1.85%
PARQUE SANTA AMÉLIA: 17 = 1.85%
PARQUE AMERICANO: 15 = 1.63%
SITIO RETIRO FELIZ: 15 = 1.63%
BABI: 13 = 1.41%
VILA ERA: 13 = 1.41%
JARDIM DAS ESTRELAS: 11 = 1.20%
RECANTUS: 9 = 0.98%
NOVA PIAM: 9 = 0.98%
VILA JOLA: 7 = 0.76%
SANTA MARIA: 6 = 0.65%
PIAM: 6 = 0.65%
DAS GRAÇAS: 5 = 0.54%
BOA ESPERANÇA: 5 = 0.54%
PRIMUS: 4 = 0.44%
NOVA AURORA: 4 = 0.44%
CENTRO: 4 = 0.44%
SANTA AMELIA: 3 = 0.33%
PAULINE: 3 = 0.33%
VILAR NOVO: 3 = 0.33%
SÃO JORGE: 2 = 0.22%
BOM PASTOR: 1 = 0.11%
BOA SORTE: 1 = 0.11%
BAIRRO DAS GRAÇAS: 1 = 0.11%
SEM TERRA: 1 = 0.11%
SAO BERNARDO: 1 = 0.11%
XAVANTES: 1 = 0.11%
VILA SÃO SEBASTIÃO: 1 = 0.11%
SHANGRI-LÁ: 1 = 0.11%
SÃO VICENTE: 1 = 0.11%
INTERLAND: 1 = 0.11%
Santa Amélia: 1 = 0.11%
Barro Vermelho: 1 = 0.11%


Segue a contagem de Plano Valor: 
99.9: 472 = 

### Como o gráfico vai ajudar?
Nos casos onde você possui dois elementos para comparar os efeitos ficam mais perceptíveis, porém o gráfico nesse caso serve para melhorar mais ainda a visibilidade para alguma apresentação futura.

In [88]:
# Comparar as contagens com os gráficos
import plotly.express as px

# Etapa 1: Criar o gráfico
for coluna in informacoes_dos_clientes:
    grafico = px.histogram(informacoes_dos_clientes, x=coluna)
# Etapa 2: Criar o gráfico
    grafico.show()

In [73]:
display(informacoes_dos_clientes.info())
display(informacoes_dos_clientes)

<class 'pandas.core.frame.DataFrame'>
Index: 919 entries, 9 to 4689
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nome/Razão Social       919 non-null    object 
 1   Data Nasc.              848 non-null    object 
 2   Data Cadastro contrato  919 non-null    object 
 3   Contrato ID             919 non-null    int64  
 4   POP                     919 non-null    object 
 5   Endereço                919 non-null    object 
 6   Bairro                  919 non-null    object 
 7   Plano                   919 non-null    object 
 8   Plano Valor             919 non-null    float64
 9   Acres.Desc.             919 non-null    float64
 10  Vencimento              919 non-null    int64  
 11  Status                  919 non-null    object 
 12  Isento                  919 non-null    object 
dtypes: float64(2), int64(2), object(9)
memory usage: 100.5+ KB


None

,Nome/Razão Social,Data Nasc.,Data Cadastro contrato,Contrato ID,POP,Endereço,Bairro,Plano,Plano Valor,Acres.Desc.,Vencimento,Status,Isento
9,ADELAIDE TEODORO TAVARES ALVES,NaN,28/04/2020,6817,FILIAL - 3-RJ,CONJUNTO CARLOS LAMARCA RUA 5 CASA N: 13,BARRO VERMELHO,4 MEGA,60.0,10.0,15,Ativo,Não
11,ADELANE OLIVEIRA DA SILVA,1994-07-01,04/09/2019,2623,FILIAL 1 E 2-RJ,RUA MADRE SILVA,BARRO VERMELHO,80_MEGA_FIBER,79.9,10.0,10,Ativo,Não
12,ADELE MARA ALMEIDA SILVA,1980-02-15,03/09/2021,4770,FILIAL 1 E 2-RJ,RUA ANAJAS,HITERLAND,250 MEGA FIBR4,99.9,10.0,20,Ativo,Não
14,ADEMIR GONÇALVES DA SILVA,1966-08-20,17/10/2022,8643,FILIAL 1 E 2-RJ,RUA ITU,PARQUE SÃO BERNARDO,SUPREMO_500MB_FIBRA_99,99.9,0.0,20,Ativo,Não
15,ADENICE MARIA DOS SANTOS,1959-07-03,11/11/2015,6170,FILIAL - 3-RJ,TRAVESSA VINTE E SETE,BARRO VERMELHO,SUPREMO_500MB_FIBRA_99,99.9,10.0,20,Ativo,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4667,WILLIAN ALVES DA SILVA,1995-08-16,13/11/2021,5069,FILIAL - 3-RJ,RUA 4 ( SEM TERRA ),BARRO VERMELHO,30 MEGA FIBRA SB,64.9,5.0,20,Ativo,Não
4674,WILSON MARTINS,1952-06-22,04/02/2022,5242,FILIAL 1 E 2-RJ,RUA TUIUTI,HITERLAND,30 MEGA FIBRA SB,64.9,0.0,5,Ativo,Não
4679,YASMIM FRANCISCA MELLO DOS SANTOS,1996-04-28,09/11/2021,4989,FILIAL 1 E 2-RJ,ESTRADA DO BARRO VERMELHO,SANTA AMÉLIA,SUPREMO_500MB_FIBRA_99,99.9,10.0,15,Ativo,Não
4681,YASMIN DE OLIVEIRA ALVES,2002-02-06,20/10/2021,1865,FILIAL - 3-RJ,RUA NATUBA,SANTA AMÉLIA,60MEGA FTTH,99.9,10.0,20,Ativo,Não


# Conclusões e Ações

Escreva aqui as suas conclusões:
Existe alguns pontos que caracterizam essas quantidade de inadimplentes, esses pontos são:
- Bairro dos clientes:
    Muitos clientes que estão afetados por essa incidência de inadimplência, estão concentrados em regiões específicas, que podem estar ligadas tanto com incidência da concorrência, quanto a condição financeira que aquelas pessoas possuem.

- plano / plano valor:
    Um valor alto surgiu nessa análise também, vale ressaltar que maior parte dos nossos clientes estão nesses planos, então talvez esse valor seja justificado por isso.
    
- Vencimento:
    A existência do vencimento no dia 20 afetou diretamente na quantidade de clientes afetados, no dia que escrevo essa análise essa data de vencimento foi retirada, porém com todos os dados apresentados fica mais que provado a conclusão.